In [2]:
import os
from dotenv import load_dotenv
import pytesseract
load_dotenv()
pytesseract.pytesseract.tesseract_cmd = r'C:\Users\ounza\AppData\Local\Programs\Tesseract-OCR\tesseract.exe'
api_key = os.environ["MISTRAL_API_KEY"]

In [3]:
from unstructured.partition.pdf import partition_pdf

file_path = 'docs/Manuel de formation - Telescope.pdf'

# Reference: https://docs.unstructured.io/open-source/core-functionality/chunking
chunks = partition_pdf(
    filename=file_path,
    infer_table_structure=True,            # extract tables
    strategy="hi_res",                     # mandatory to infer tables

    extract_image_block_types=["Image"],   # Add 'Table' to list to extract image of tables
    # image_output_dir_path=output_path,   # if None, images and tables will saved in base64

    extract_image_block_to_payload=True,   # if true, will extract base64 for API usage

    chunking_strategy="by_title",          # or 'basic'
    max_characters=10000,                  # defaults to 500
    combine_text_under_n_chars=2000,       # defaults to 0
    new_after_n_chars=6000,

    # extract_images_in_pdf=True,          # deprecated
)

c:\Users\ounza\IMT\Semestre_6\PRONTO\ai-agent\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
# separate tables from texts
tables = []
texts = []

for chunk in chunks:
    if "Table" in str(type(chunk)):
        tables.append(chunk)

    if "CompositeElement" in str(type((chunk))):
        texts.append(chunk)

# Get the images from the CompositeElement objects
def get_images_base64(chunks):
    images_b64 = []
    for chunk in chunks:
        if "CompositeElement" in str(type(chunk)):
            chunk_els = chunk.metadata.orig_elements
            for el in chunk_els:
                if "Image" in str(type(el)):
                    images_b64.append(el.metadata.image_base64)
    return images_b64

images = get_images_base64(chunks)

In [8]:
from langchain_mistralai.chat_models import ChatMistralAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import time

In [9]:
text_model = ChatMistralAI(mistral_api_key=api_key,model="mistral-large-latest")
prompt_text = """
You are an assistant tasked with summarizing tables and text.
Give a concise summary of the table or text.

Respond only with the summary, no additionnal comment.
Do not start your message by saying "Here is a summary" or anything like that.
Just give the summary as it is.

Table or text chunk: {element}

"""
prompt = ChatPromptTemplate.from_template(prompt_text)
summarize_chain = {"element": lambda x: x} | prompt | text_model | StrOutputParser()

In [10]:
text_summaries = []
def summarize_element(item, summarize_chain):
    return summarize_chain.invoke({"element": item})

for chunk in texts:
    try:
        summary = summarize_element(chunk, summarize_chain)
        text_summaries.append(summary)
        time.sleep(2)
    except Exception as e:
        print(f"Failed to summarize text chunk: {e}")
        text_summaries.append(None)

In [11]:
# Summarize tables
tables_html = [table.metadata.text_as_html for table in tables]
table_summaries = []
for html in tables_html:
    try:
        summary = summarize_element(html, summarize_chain)
        table_summaries.append(summary)
        time.sleep(2)
    except Exception as e:
        print(f"Failed to summarize table chunk: {e}")
        table_summaries.append(None)


In [17]:
from mistralai import Mistral

client = Mistral(api_key=api_key) 
image_summaries = []
for image in images:
    prompt_template = """Describe the image in detail. For context,
                    L'image est extraite de la documentation du matériel utilisé dans l'observatoire astronomique de la Pointe du Diable, géré par l'association Gens de la Lune."""
    messages = [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": prompt_template},
                {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{image}"}}
            ]
        }
    ]
    
    # Perform inference
    response = client.chat.complete(
        model="pixtral-large-latest",
        messages=messages,
        max_tokens=300
    )
    # Return the model's output
    print(response.choices[0].message.content)
    image_summaries.append(response.choices[0].message.content)
    time.sleep(2)



Cette image montre l'observatoire astronomique de la Pointe du Diable, un lieu dédié à l'observation des astres. Voici une description détaillée :

1. **Bâtiment de l'Observatoire** :
   - **Dôme** : Le bâtiment est surmonté d'un dôme blanc qui abrite probablement le télescope principal de l'observatoire.
   - **Structure** : Il s'agit d'un bâtiment rectangulaire avec des murs blancs et une ouverture rectangulaire près du dôme, qui peut être utilisée pour l'observation ou pour des ajustements techniques.

2. **Environnement** :
   - **Paysage** : L'observatoire est situé dans une zone rurale, entourée d'arbres et de verdure. La silhouette des arbres est visible devant le bâtiment.
   - **Atmosphère** : L'image est prise pendant la nuit, ce qui est typique pour un observatoire, car les astronomes préfèrent observer le ciel nocturne pour éviter l'interférence de la lumière du soleil.

3. **Ciel Nocturne** :

L'image représente le logo de l'association "Gens de la Lune". Voici une descrip